In [1]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 954.5 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 68.9 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 74.2 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3


In [17]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.
input_texts = ['query: how much protein should a female eat',
               'query: 南瓜的家常做法',
               "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
               "passage: 1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅"]

tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large').to('cuda')

# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt').to('cuda')

outputs = model(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:2] @ embeddings[2:].T) * 100
print(scores.tolist())


[[90.81391906738281, 72.1339111328125], [70.5354232788086, 88.76110076904297]]


In [11]:
import pandas as pd
import numpy as np
import torch

books = pd.read_csv('/kaggle/input/nto-ai-2stage-individual/books.csv')
desc = pd.read_csv('/kaggle/input/nto-ai-2stage-individual/book_descriptions.csv')
books = books.merge(desc, on='book_id', how='left')

In [12]:
books['text'] = "Title: " + books['title'] + "\nDescription: " + books['description']

In [13]:
print(books['text'].iloc[1])

Title: Самое жуткое приключение
Description: Когда Ларри Бойд и его друзья решили применить лосьон "Мгновенный загар", они не предполагали, чем это может закончиться...


In [14]:
class TextData(torch.utils.data.Dataset):
    def __init__(self, texts):
        self.texts = texts
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        return self.texts[idx]

In [15]:
dl = torch.utils.data.DataLoader(TextData(books['text'].values), batch_size=100, shuffle=False)

In [28]:
np.quantile([len(x.split()) for x in books['text']], 0.95)

186.0

In [30]:
from tqdm.auto import tqdm

embs = []
for text in tqdm(dl):
    tokenized_inputs = tokenizer(text, max_length=200, padding=True, truncation=True, return_tensors="pt").to('cuda')
    
    with torch.no_grad():
        outputs = model(**tokenized_inputs)
        
    embeddings = average_pool(
        outputs.last_hidden_state, 
        tokenized_inputs["attention_mask"],
        # pooling_method="mean"
    )
    embs.append(embeddings.cpu())

  0%|          | 0/505 [00:00<?, ?it/s]

In [33]:
embeddings = F.normalize(torch.cat(embs).cpu(), p=2, dim=1)

In [35]:
np.save('e5-large_embs_books.npy', embeddings.numpy())